<a href="https://colab.research.google.com/github/hamzafarooq/algos/blob/code/dnn_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Package imports 
import numpy as np 
import matplotlib.pyplot as plt 
# here planar_utils.py can be found on its github repo 
import sklearn
from sklearn import datasets



In [ ]:

def plot_decision_boundary(model, X, y):
    # Set min and max values and give it some padding
    x_min, x_max = X[0, :].min() - 1, X[0, :].max() + 1
    y_min, y_max = X[1, :].min() - 1, X[1, :].max() + 1
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole grid
    Z = model(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.ylabel('x2')
    plt.xlabel('x1')
    plt.scatter(X[0, :], X[1, :], c=y, cmap=plt.cm.Spectral)
    

def sigmoid(x):
    """
    Compute the sigmoid of x
    Arguments:
    x -- A scalar or numpy array of any size.
    Return:
    s -- sigmoid(x)
    """
    s = 1/(1+np.exp(-x))
    return s

def load_planar_dataset():
    np.random.seed(1)
    m = 400 # number of examples
    N = int(m/2) # number of points per class
    D = 2 # dimensionality
    X = np.zeros((m,D)) # data matrix where each row is a single example
    Y = np.zeros((m,1), dtype='uint8') # labels vector (0 for red, 1 for blue)
    a = 4 # maximum ray of the flower

    for j in range(2):
        ix = range(N*j,N*(j+1))
        t = np.linspace(j*3.12,(j+1)*3.12,N) + np.random.randn(N)*0.2 # theta
        r = a*np.sin(4*t) + np.random.randn(N)*0.2 # radius
        X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
        Y[ix] = j
        
    X = X.T
    Y = Y.T

    return X, Y

def load_extra_datasets():  
    N = 200
    noisy_circles = sklearn.datasets.make_circles(n_samples=N, factor=.5, noise=.3)
    noisy_moons = sklearn.datasets.make_moons(n_samples=N, noise=.2)
    blobs = sklearn.datasets.make_blobs(n_samples=N, random_state=5, n_features=2, centers=6)
    gaussian_quantiles = sklearn.datasets.make_gaussian_quantiles(mean=None, cov=0.5, n_samples=N, n_features=2, n_classes=2, shuffle=True, random_state=None)
    no_structure = np.random.rand(N, 2), np.random.rand(N, 2)
    
    return noisy_circles, noisy_moons, blobs, gaussian_quantiles, no_structure

In [ ]:
X.shape

(569, 30)

In [ ]:
bc = datasets.load_breast_cancer()
X,Y = load_planar_dataset()

In [ ]:
Y.shape[1]

400

In [ ]:
#Reshape mmatrix from a 2x3 array  to a 3x2 array
mmatrix= np.array([[1,2,3],[4,5,6]])
print(mmatrix)
rematrix = mmatrix.reshape(3,2)
print(rematrix.shape)
output:(3, 2)
#flatten mmatrix to 1-d array
flmatrix = mmatrix.flatten()
print(flmatrix)


[[1 2 3]
 [4 5 6]]
(3, 2)
[1 2 3 4 5 6]


In [ ]:
import math
def relu(X):
   return np.maximum(0,X)

def sigmoid(X):
   return 1/(1+np.exp(-X))

#softmax function
def softmax(X):
    expo = np.exp(X)
    expo_sum = np.sum(np.exp(X))
    return expo/expo_sum
#Example with mmatrix defined above
print (softmax(mmatrix))
print (sigmoid(mmatrix))
print (relu(mmatrix))


[[0.00426978 0.01160646 0.03154963]
 [0.08576079 0.23312201 0.63369132]]
[[0.73105858 0.88079708 0.95257413]
 [0.98201379 0.99330715 0.99752738]]
[[1 2 3]
 [4 5 6]]


In [ ]:
W1 = np.random.randn(4, X.shape[0]) * 0.01
b1 = np.zeros(shape =(4, 1)) 
  
W2 = np.random.randn(Y.shape[0], 4) * 0.01
b2 = np.zeros(shape =(Y.shape[0], 1)) 


In [ ]:
W2

array([[ 0.00355717,  0.00849587, -0.00122154, -0.00680852]])

In [ ]:
def forward_prop(X, W1, W2, b1, b2): 
  
    Z1 = np.dot(W1, X) + b1 
    A1 = np.tanh(Z1) 
    Z2 = np.dot(W2, A1) + b2 
    A2 = sigmoid(Z2) 
      
    # here the cache is the data of previous iteration 
    # This will be used for backpropagation 
    cache = {"Z1": Z1, 
             "A1": A1, 
             "Z2": Z2, 
             "A2": A2} 
      
    return A2, cache 

In [ ]:
# Here Y is actual output 
def compute_cost(A2, Y): 
    m = Y.shape[1]
    # implementing the above formula 
    cost_sum = np.multiply(np.log(A2), Y) + np.multiply((1 - Y), np.log(1 - A2)) 
    cost = - np.sum(cost_sum) / m 
      
    # Squeezing to avoid unnecessary dimensions 
    cost = np.squeeze(cost) 
    return cost 

In [ ]:
def back_propagate(W1, b1, W2, b2, cache): 
    m = X.shape[1]
    # Retrieve also A1 and A2 from dictionary "cache" 
    A1 = cache['A1'] 
    A2 = cache['A2'] 
    learning_rate = 0.001
    # Backward propagation: calculate dW1, db1, dW2, db2.  
    dZ2 = A2 - Y 
    dW2 = (1 / m) * np.dot(dZ2, A1.T) 
    db2 = (1 / m) * np.sum(dZ2, axis = 1, keepdims = True) 
  
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2)) 
    dW1 = (1 / m) * np.dot(dZ1, X.T) 
    db1 = (1 / m) * np.sum(dZ1, axis = 1, keepdims = True) 
      
    # Updating the parameters according to algorithm 
    W1 = W1 - learning_rate * dW1 
    b1 = b1 - learning_rate * db1 
    W2 = W2 - learning_rate * dW2 
    b2 = b2 - learning_rate * db2 
  
    return W1, W2, b1, b2 

In [ ]:
for i in range(0, 1000): 
    
        # Forward propagation. Inputs: "X, parameters". return: "A2, cache". 
        A2, cache = forward_prop(X, W1, W2, b1, b2) 
          
        # Cost function. Inputs: "A2, Y". Outputs: "cost". 
        cost = compute_cost(A2, Y) 
        
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads". 
        W1, W2, b1, b2 = back_propagate(W1, b1, W2, b2, cache) 
          
        # Print the cost every 1000 iterations 
        if cost and i % 1000 == 0: 
            print ("Cost after iteration % i: % f" % (i, cost))

Cost after iteration  0:  0.681660


In [ ]:
A2

array([[0.49594505, 0.49596406, 0.49597052, ..., 0.499836  , 0.49964744,
        0.49989463],
       [0.4942951 , 0.4942338 , 0.49420019, ..., 0.499843  , 0.49961718,
        0.49989695],
       [0.49769432, 0.4976961 , 0.49769496, ..., 0.49987596, 0.49971906,
        0.49991504],
       ...,
       [0.50508838, 0.50517481, 0.50522302, ..., 0.50010792, 0.50029709,
        0.50007475],
       [0.50187112, 0.50182296, 0.50179565, ..., 0.50014943, 0.5002921 ,
        0.50009368],
       [0.49815161, 0.49821032, 0.49823912, ..., 0.49983373, 0.49966305,
        0.49989042]])